In [ ]:
#r "nuget: Plotly.NET, 3.0.0"
#r "nuget: Plotly.NET.Interactive, 3.0.0"
#r "nuget: Microsoft.Data.Analysis, 0.18.0"

In [ ]:

open System.IO
open System.Diagnostics
open Microsoft.Data.Analysis
open Plotly.NET
open Newtonsoft.Json
open Newtonsoft.Json.Linq

let run name args workingDir =
    let info = ProcessStartInfo()
    info.WindowStyle <- ProcessWindowStyle.Hidden
    info.Arguments <- args
    info.FileName <- name
    info.UseShellExecute <- false
    info.WorkingDirectory <- workingDir
    info.RedirectStandardError <- true
    info.RedirectStandardOutput <- true
    info.RedirectStandardInput <- true
    info.CreateNoWindow <- true
    printfn $"Run {name} {args} in {workingDir}"
    let p = Process.Start(info)
    let o = p.StandardOutput.ReadToEnd()
    let errors = p.StandardError.ReadToEnd()
    p.WaitForExit()
    printfn "%A" o
    if p.ExitCode <> 0 then
        failwith $"Process {name} {args} failed: {errors}."

type FCSVersion =
    | Local
    | NuGet of string
    | Git of string
    override this.ToString() =
        match this with
        | Local -> "local"
        | NuGet version -> $"v{version}"
        | Git revision -> $"git_{revision}"

let getOutputDir (version : FCSVersion) =
    Path.Combine(__SOURCE_DIRECTORY__, $".artifacts/{version}")

let readResultsJson (version : FCSVersion) =
    let baseDir = getOutputDir version
    let resultsPath = Path.Combine(baseDir, @"results/HistoricalBenchmark.DecentlySizedStandAloneFileBenchmark-report.json")
    let json = File.ReadAllText(resultsPath)
    JsonConvert.DeserializeObject<JObject>(json)

let runBenchmark (version : FCSVersion) (msBuildArgs : (string * string) list) =
    msBuildArgs
    |> List.iter (fun (key, value) ->
        Environment.SetEnvironmentVariable(key, value)
    )
    run "dotnet" $"build -c Release HistoricalBenchmark.fsproj" __SOURCE_DIRECTORY__
    let outputDir = getOutputDir version
    run "dotnet" $"run -c Release --no-build --project HistoricalBenchmark.fsproj -- --filter *DecentlySizedStandAloneFileBenchmark* -a {outputDir}" __SOURCE_DIRECTORY__

let benchmarkCurrent (name : string) =
    let version = FCSVersion.Local
    printfn $"Benchmarking '{version}'..."
    let msBuildArgs =
        [
            "FcsReferenceType", "project"
            "FcsProjectPath", @"..\..\..\..\src\Compiler\FSharp.Compiler.Service.fsproj"
            "DefineConstants", ""
        ]
    runBenchmark version msBuildArgs
    printfn $"'{version}' Done"
    name, version

let benchmarkVersion (name: string) (version: string) (constants: string) =
    let fcsVersion = FCSVersion.NuGet version
    printfn $"Benchmarking '{version}'..."
    let msBuildArgs =
        [
            "FcsReferenceType", "nuget"
            "FcsNuGetVersion", version
            "DefineConstants", constants
        ]
    runBenchmark fcsVersion msBuildArgs
    printfn $"'{version}' Done"
    name, fcsVersion

[<RequireQualifiedAccess>]
type UseVisualStudio =
    | Yes
    | No

let benchmarkCommit (commitHash: string) (useVisualStudio : UseVisualStudio) =
    let version = FCSVersion.Git commitHash
    printfn $"Benchmarking '{version}'..."
    let tmp = Path.GetTempFileName()
    try File.Delete(tmp) with | _ -> ()
    let tmpDir = Path.Combine(Path.GetDirectoryName(tmp), Path.GetFileNameWithoutExtension(tmp))
    Directory.CreateDirectory(tmpDir) |> ignore

    try
        let fcsOutputPath = Path.Combine(tmpDir, "artifacts/bin/FSharp.Compiler.Service/Release/netstandard2.0/FSharp.Compiler.Service.dll")

        printfn $"Cloning 'dotnet/fsharp.git' in '{tmpDir}'..."
        run "git" $"clone https://github.com/dotnet/fsharp.git {tmpDir}" __SOURCE_DIRECTORY__
        printfn $"Switching to '{commitHash}'..."
        run "git" $"reset --hard {commitHash}" tmpDir
        printfn "Building fsharp..."
        let vsArg = match useVisualStudio with | UseVisualStudio.Yes -> "" | UseVisualStudio.No -> " -noVisualStudio"
        run "cmd" $"/C build.cmd -c Release {vsArg}" tmpDir
        
        printfn $"Benchmarking {version}..."
        let msBuildArgs =
            [
                "FcsReferenceType", "dll"
                "FcsDllPath", fcsOutputPath
                "DefineConstants", ""
            ]
        runBenchmark version msBuildArgs

        printfn $"'{version}' Done"
        commitHash, version
    finally
        try Directory.Delete(tmpDir) with | _ -> ()

In [ ]:
let benchmarkCommit' commit = benchmarkCommit commit UseVisualStudio.No
let runs =
    [
      // Note: SERVICE_30_0_0 and SERVICE_13_0_0 refer to define constants in order to build the benchmark on older versions.
      benchmarkCurrent "local"

      benchmarkCommit' "a901fe2862dce0644ac8104d24e51e664b2d553f"
      //benchmarkCommit' "81d1d918740e9ba3cb2eb063b6f28c3139ca9cfa"
      //benchmarkCommit' "1d36c75225436f8a7d30c4691f20d6118b657fec"
      //benchmarkCommit' "2e4096153972abedae142da85cac2ffbcf57fe0a"
      //benchmarkCommit' "af6ff33b5bc15951a6854bdf3b226db8f0e28b56"
      
      benchmarkVersion "41.0.5 (6/14/2022)" "40.0.0" ""
      benchmarkVersion "40.0.0 (6/22/2021)" "40.0.0" ""
      benchmarkVersion "35.0.0 (4/10/2020)" "35.0.0" "SERVICE_30_0_0"
      benchmarkVersion "30.0.0 (6/29/2019)" "30.0.0" "SERVICE_30_0_0"
      benchmarkVersion "25.0.1 (9/5/2018)" "25.0.1" "SERVICE_13_0_0"
      benchmarkVersion "20.0.1 (2/21/2018)" "20.0.1" "SERVICE_13_0_0"
      benchmarkVersion "13.0.0 (6/28/2017)" "13.0.0" "SERVICE_13_0_0"
    ]

In [ ]:
let extractResultsData (results : JObject) =
    let benchmark = results["Benchmarks"][0]
    let stats = benchmark["Statistics"]
    let mean = stats["Mean"].ToString() |> Double.Parse
    let metrics = benchmark["Metrics"] :?> JArray
    let am =
        metrics
        |> Seq.find (fun m -> (m["Descriptor"]["Id"]).ToString() = "Allocated Memory")
        |> fun m -> m["Value"].ToString() |> Double.Parse
        
    mean, am

let fullResults =
    runs
    |> List.map (fun (name, version) -> name, readResultsJson version)
let results =
    fullResults
    |> List.map (fun (name, fullResult) ->
        name, extractResultsData fullResult
    )

let x =
     results
     |> List.map (fun (name, _) -> name)
let y1 =
    results
    |> List.map (fun (name, (mean, allocated)) -> Math.Round(mean / 1000000.0, 2))
let meanData = (x, y1) ||> List.zip |> List.rev
let y2 =
    results
    |> List.map (fun (name, (mean, allocated)) -> Math.Round(allocated / 1024.0 / 1024.0, 2))
let allocatedData = (x, y2) ||> List.zip |> List.rev

// Charts

let meanLine = Chart.Line(meanData, Name="Mean (ms)")
let allocatedLine = Chart.Line(allocatedData, Name="Allocated (MB)")

Chart.combine([meanLine;allocatedLine])